In [ ]:
from kfold_and_metrics import *

import tensorflow as tf
import pandas as pd
import numpy as np

tf.keras.backend.clear_session()

In [ ]:
df = pd.read_csv("final.csv")
df = df.drop(columns=['id'])
df.head()

In [ ]:
activation_possibilities = ["softmax", "relu", "sigmoid"]
max = 0
best = []
for activation_hidden in activation_possibilities:
    for num_nodes_hidden_layer in range(5, len(df.columns)*2//3, 10):
        for gradient in [0.001, 0.003, 0.005, 0.007, 0.01]:
            print(f"Current parameter combination:\n\tNumber of Hidden Layers: {num_nodes_hidden_layer}\n\tActivation Function for the Hidden Layer: {activation_hidden}\n\tLearning Rate: {gradient}")
            nn_model = tf.keras.models.Sequential([
                tf.keras.layers.Input((len(df.columns)-2,), name="input"),
                tf.keras.layers.Dense(num_nodes_hidden_layer,activation=activation_hidden),
                tf.keras.layers.Dense(2,activation='softmax')
            ])
            nn_model.compile(
                optimizer=tf.keras.optimizers.SGD(gradient), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            )
            score = k_fold_cv_keras(compiled_model=nn_model, df=df)
            results = mean_std_results_k_fold_CV(score)

            print(results)

            accuracy_avg = results.iloc[1,1]
            if accuracy_avg > max:
                best = [num_nodes_hidden_layer, activation_hidden, gradient]

print(f"Number of Hidden Layers: {best[0]}\nActivation Function for the Hidden Layer: {best[1]}\nLearning Rate: {best[2]}")